In [1]:
import numpy as np
import cv2 as cv
import os
from matplotlib import pyplot as plt
from glob import glob
import skimage
from skimage import measure, color, io
import scipy
import pandas as pd
from tifffile import imread
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
import sys
import tensorflow as tf

%matplotlib inline

In [11]:
X = sorted(glob("/home/devesh/cell-segmentation/images/*.tif"))
X = list(map(imread,X))
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

Normalizing image channels independently.


In [14]:
n_channel

3

In [5]:
demo_model = True

if demo_model:
    print (
        "NOTE: This is loading a previously trained demo model!\n"
        "      Please set the variable 'demo_model = False' to load your own trained model.",
        file=sys.stderr, flush=True
    )
    model = StarDist2D.from_pretrained('2D_demo')
else:
    model = StarDist2D(None, name='stardist', basedir='models')
None;

NOTE: This is loading a previously trained demo model!
      Please set the variable 'demo_model = False' to load your own trained model.


Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


2022-07-26 15:32:38.103616: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
X[1].shape


(1840, 1840, 3)

In [6]:
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

In [13]:
axis_norm = (0,1)
img = normalize(X[1], 1,99.8, axis=axis_norm)
labels, details = model.predict_instances(img)

ValueError: 